In [1]:
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
chat = ChatOpenAI()
memory = ConversationBufferMemory()

In [ ]:
prompt = PromptTemplate.from_template(
"""
Você é um grande leitor de livros e conselheiro tbm.\
Tem um tom ironico e sacadas que lembra Abujamra.\
Não precisa ficar falando tanto, mas você é uma pessoa interessante.\
Seu nome é Bonifácio e sabe tudo sobre literatura, mas sabe conversar sobre qualquer assunto.\
você trabalhou em várias bibliotecas, depois começou a dar palestras e ficou rico.\
Hoje você está morando na Finlandia.\

Conversa atual:

{history}

Human: {input}
AI:
"""
)

In [ ]:
chain = ConversationChain(
    llm=chat,
    memory=memory,
    prompt=prompt
)

In [ ]:
chain.predict(input='onde vc mora e qual autor q eu gosto?')

**LLMCHAIN**

In [ ]:
from langchain.chains.llm import LLMChain

In [ ]:
prompt_template = PromptTemplate.from_template(
"""
Você vai receber um nome comum e vai transformá-lo em apelido do futebol brasileiro\
use nomes e apelidos comumente usados para jogadores no Brasil. Pode ter ligação com \
o nome, mas não necessariamente. Este é o nome: {nome}
""")

chain_apelido = LLMChain(
    llm=chat,
    prompt=prompt_template,
    
)

prompt_frase = PromptTemplate.from_template(
"""
O narrador da partida precisa apresentar com emoção!\
Crie uma frase épica e um bordão pra narrar o gol do jogador {apelido}
""")

chain_frase = LLMChain(
    llm=chat,
    prompt=prompt_frase,
    
)

prompt_grito = PromptTemplate.from_template(
"""
Agora você vai simular o comentário do assistente depois da narração. {frase}\
Faça um complemento interessante, digno de grandes comentaristas.
""")

chain_grito = LLMChain(
    llm=chat,
    prompt=prompt_grito,
    
)

**SIMPLE SEQUENTIAL CHAIN**

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
chain_final = SimpleSequentialChain(
    chains=[chain_apelido, chain_frase, chain_grito],
    verbose=True # aqui ele vai mostrar todas cadeias de pensamento
)

In [ ]:
nome = "Baiano"

chain_final.run(nome)

**SEQUENTIAL CHAIN**

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
prompt_template = PromptTemplate.from_template(
"""
Você terá um assunto para debater. Não precisa de uma argumentação longa.
Só precisa dar uma opinião bem polemica sobre o tema. Este é o tema: {tema}
""")

chain_apelido = LLMChain(
    llm=chat,
    prompt=prompt_template,
    output_key='opiniao1'
    
)

prompt_frase = PromptTemplate.from_template(
"""
Dado o tema {tema} e a opinião polemica do colega: "{opiniao1}".
Agora desenvolda uma replica refutando essa opiniao do colega.
""")

chain_frase = LLMChain(
    llm=chat,
    prompt=prompt_frase,
    output_key='opiniao2'
    
)

prompt_coment = PromptTemplate.from_template(
"""
Seu papel aqui é o da síntese. Dado o tema {tema} e 
as opiniões colega 1: "{opiniao1}". Analise tambem a opinião do colega 2: "{opiniao2}".
Agora crie um meio termo para que ambos possam chegar num acordo.
""")

chain_grito = LLMChain(
    llm=chat,
    prompt=prompt_coment,
    output_key='sintese'
)

chain = SequentialChain(
    chains=[chain_apelido, chain_frase, chain_grito],
    input_variables=['tema'],
    output_variables=['opiniao1', 'opiniao2', 'sintese'],
    verbose=True
)

nome='futebol'

chain.invoke({'tema':nome})

**RUNNABLES**

São componentes especiais do langchain. Eles são modulares, facilitam a execução de tarefas, reutilização do código e composição/encademaneto de fluxos de ia.

Vc pode criar funções de python com runnables e a partir desse momento as funções adquirem os métodos dos runnables como invoke, stream, batch...

Vamos dar um exemplo usando a o RunnalbleLambda:    

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel

# Criamos um Runnable que transforma um texto para minúsculas
minusculo = RunnableLambda(lambda x: x.lower())

# Testando a transformação
minusculo.invoke('FALA COMIGO!')

In [ ]:
# veja q o runnable lambda tbm aceita função nomeada

def converte_minusculo(palavra)-> str:
    return palavra.lower()

# Criamos um Runnable que transforma um texto para minúsculas
minusculo = RunnableLambda(converte_minusculo)

# Testando a transformação
minusculo.invoke('FALA COMIGO!')

Agora vamos para outro runnable um pouco mais elaborado, o RunnableParallel.

Ele pode rodar chains em paralelo de forma assíncrona.

Podemos fazer um encadeamento para que o modelo gere alguns outputs iniciais e um modelo final consuma esse output, facilitando o fluxo.

Vamos ver isso em código:

In [ ]:
# vamos criar duas chains
# e depois vamos rodá-las em paralelo

prompt_nome_produto = PromptTemplate.from_template(
    """
Vc vai desenvolver um nome para o seguinte tipo de produto: {produto}.
Deve ter um apelo de marketing interessante!
"""
)

chain_produto = prompt_nome_produto | chat


prompt_cliente = PromptTemplate.from_template(
    """
Vc é um publicitário e vai me dizer qual é o perfil do público em potencial
desse produto: {produto}
"""
)

chain_cliente = prompt_cliente | chat

In [ ]:
# criando um runnable das chains
# o output é um dicionário e cada chave
# traz o retorno de sua respectiva chain
paralelo = RunnableParallel(nome_produto=chain_produto, publico=chain_cliente)

paralelo.invoke('cachaça')

In [ ]:
# esse prompt vai receber o output das chains anteriores, executadas em paralelo

prompt_peca = PromptTemplate.from_template(
    """
Vc é um publicitário rebuscado e muito conceituado.

Vc vai desenvolver uma peça publicitária, um chamado para 
o público consumir o produto. Vou te falar o nome da empresa e 
a descrição do público alvo. Segue:

nome da empresa: {nome_produto}
publico alvo: {publico}
"""
)

In [ ]:
# aqui nós juntamos tudo através dos pipes
# o paralelo fornece os argumentos para o prompt_peca
# esse prompt final é fornecido para um modelo
# e depois o output parser traz formata esse retorno para exibição
chain_final = paralelo | prompt_peca | chat | StrOutputParser()

In [ ]:
# aqui chamamos o metodo invoke para ver o resultado
chain_final.invoke({'produto':'reports e analise de dados para o setor de RH'})

**CADEIAS DE ROTEAMENTO**

O roteamento permite direcionar a demanda do usuário (pergunta ou algo tipo) para a chain especializada em responder aquela pergunta. 

Por exemplo, podemos ter um aplicativo que receba perguntas de alunos do ensino fundamental.

Com o roteador, ele consegue categorizar essa pergunta e direcionar para a chain correta.

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
# instanciando o modelo
model = ChatOpenAI(model='gpt-4o-mini')

# criando os prompts especializados para cada matéria
fis_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de física.
Vc é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

chain_fisica = fis_template | model

mat_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de matemática, a verdadeira reencarnação de Pitágoras.\
Vc é ótimo em responder perguntas sobre matemática de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

chain_mat = mat_template | model

hist_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de história, a verdadeira reencarnação de Eric Hobsbawm.\
Vc é ótimo em responder perguntas sobre história de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

chain_hist = hist_template | model

# esse aqui é um prompt genérico de escape
# caso a pergunta do aluno nao se encaixe em nenhuma alternativa
prompt_generico = ChatPromptTemplate.from_template('''{pergunta}''')
chain_generica = prompt_generico | model

Agora, vamos criar um "parseador" para que ele retorne a resposta exatamente da forma que precisamos. Para isso, vamos utilizar o pydantic.

In [ ]:
# vamos criar um Categorizador, que será um objeto Pydantic

class Categorizador(BaseModel):
    "Categoriza as perguntas dos alunos"
    area_conhecimento: str = Field(description="A área de conhecimento \
                                   da pergunta feita pelo aluno. Deve ser:\
                                   'física', 'matemática' ou 'história'.\
                                   Caso não encontre nenhuma delas, retorne 'outra'.")
    

# vamos adicionar este prompt à chain
# não é obrigatório, mas pode ajudar 
# a retornar melhores respostas do modelo
    
prompt_categoria = ChatPromptTemplate.from_template("Você deve categorizar\
                                                    a seguinte pergunta: {pergunta}.\
                                                    Vc deve retornar sempre nesse padrão,\
                                                    com letras em minúculas.")
    
# criando a chain
model_estruturado = prompt_categoria | model.with_structured_output(Categorizador)

In [ ]:
# testando
model_estruturado.invoke({'pergunta':"quando foi o periodo da ditadura no brasil?"})

CRIANDO ESTRUTURA DE ROTEAMENTO

O roteador será uma função, q vai pegar o outuput do parseador q criamos acima, e chamar e atribuir para a chain necessária.

Antes, vamos criar um runnable para agilizar a interação com esta função.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# aqui só uma demontração do q esse runnable faz
# veja que ele recebe a pergunta e a resposta e 
# passa para frente, em forma de dicionário
chain_runnable = RunnablePassthrough().assign(categoria=model_estruturado)
chain_runnable.invoke({'pergunta':'quem foi Ruy Barbosa?'})

In [ ]:
# vamos agor definir a função roteadora
# será uma condicional simples
def route(input):
    if input['categoria'] == 'matemática':
        return chain_mat
    if input['categoria'] == 'física':
        return chain_fisica
    if input['categoria'] == 'história':
        return chain_hist
    
    
    return chain_generica

In [ ]:
# agora vamos fazer nossa chain com runnable e agora com o roteador
chain_runnable = RunnablePassthrough().assign(categoria=model_estruturado) | route

In [ ]:
# veja que ele recebe a pergunta e responde de acordo
# as chains especializadas poderiam ser com outras funções e mais complexas
# o roteamento permite que a gente faça essa distribuição de forma mais eficiente
chain_runnable.invoke({'pergunta':'quem foi Ruy Barbosa?'})

### MEMÓRIA

Sem memória, nosso modelo não tem nenhum contexto da conversa. Toda mensagem será uma nova mensagem sem histórico algum.

Aqui vamos aprender a adicionar memória ao nosso bot usando langchain.

Vamos usar a classe InMemoryChatMessageHistory. Ela guarda uma lista de mensagens (self.messages), mas só em memória RAM — ou seja, não persiste no disco ou banco de dados. Quando o processo termina (por exemplo, se você fechar o servidor ou recarregar o notebook), tudo é perdido.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [ ]:
# instanciando a memoria
memory = InMemoryChatMessageHistory()

In [ ]:
# podemos usar alguns métodos dessa classe
# para adicionar mensagens ao histórico
memory.add_user_message('olá, meu nome é Kaio.')
memory.add_ai_message('olá, guardei seu nome.')

In [ ]:
# e aqui podemos visualizar esse histórico
memory.messages

In [ ]:
# aqui vamos construir uma chain pronta para receber a memoria
# para esse fim, é necessário criar esse placeholder conforme abaixo
# e atribuir o nome 'history'

prompt = ChatPromptTemplate.from_messages([
    ('system', 'você é um tutor de programação q se chama Lovelace.\
     Responda às perguntas de forma didatica.'),
     ('placeholder', '{history}'),
     ('human', '{pergunta}')
])

# criando a chain
chain = prompt | ChatOpenAI()

Agora vamos criar a memória para adicionar ao modelo.

In [ ]:
# criando um dicionário onde ficarão as conversas por usuário
store = {}

# e aqui criamos uma função para adicionar 
# nova seção de memória ao dicionário
# caso já exista, ele busca e retorna essa memória 
# cada usuário (ou sessão) tem seu próŕio histórico
def get_session_by_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


Agora vamos adicionar a memoria criada anteriormente à nossa chain, que tbm já havíamos criado. Para isso, utilizaremos um runnable específico para este fim.

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# agora usando o runnable para criar a chain final
# com memoria

chain_com_memoria = RunnableWithMessageHistory(
    chain, # fornecendo a chain com o prompt e modelo
    get_session_by_id, # fornecendo a função q gerencia memoria
    input_messages_key='pergunta', # fornecendo a variavel de entrada do usuário
    history_messages_key='history' # fornecendo a variavel dos historico de conversa
)

Agora vamos testar essa nova chain com memoria.

Antes de testar a nova chain com memória, precisamos criar um dicionário chamado config, que será responsável por gerenciar diferentes usuários através de um identificador único (session_id). Esse session_id é essencial para que a memória funcione corretamente, permitindo que o histórico de conversas seja separado por usuário. Abaixo, configuramos o session_id como "user_a" e chamamos a chain passando tanto a pergunta quanto a configuração:

In [ ]:
config = {'configurable': {'session_id':'user_a'}}
resposta = chain_com_memoria.invoke({'pergunta':'olá meu nome é Zé!'},
                                    config=config)
resposta

In [ ]:
# vamos só validar se ele manteve a memoria e lembra o meu nome
resposta = chain_com_memoria.invoke({'pergunta':'qual é o meu nome?'},
                                    config=config)
resposta

In [ ]:
# agora olha q coisa interessante
# veja que se mudar de usuário no config
# ele já muda de histórico
# agora o modelo não sabe mais o nome do usuário pq esse de fato é outro usuario
config = {'configurable': {'session_id':'user_b'}}
resposta = chain_com_memoria.invoke({'pergunta':'qual é o meu nome?'},
                                    config=config)
resposta

## RAG: Conversando com os seus dados

### 🔍 Lembrete: Retrieval-Augmented Generation (RAG)

**O que é RAG?**
- Técnica que combina **geração de texto (LLM)** com **recuperação de informações externas**.
- Permite que o modelo acesse dados atualizados e específicos que não estão em seu treinamento original.

**Por que é importante?**
- LLMs sozinhos dependem apenas do conhecimento aprendido até a data de corte.
- RAG permite incorporar informações externas (PDFs, documentos, banco de dados etc.).
- Ideal para criar **aplicações personalizadas**, como chatbots que respondem com base em documentos internos.

**Etapas do RAG:**
1. **📥 Carregamento de Documentos**: importar PDFs, CSVs, bancos de dados, etc.
2. **✂️ Divisão de Documentos**: separar em trechos menores mantendo o contexto.
3. **🔢 Embedding**: transformar os trechos em vetores numéricos.
4. **💾 Armazenamento em VectorStore**: salvar os vetores para busca eficiente.
5. **🔍 Recuperação (Retrieval)**: buscar os vetores mais relevantes com base na pergunta do usuário.
6. **🧠 Geração**: o modelo usa os trechos recuperados para gerar uma resposta precisa.

**Desafios:**
- Limite de tokens nos modelos (ex: 16k no GPT-3.5).
- Selecionar apenas os trechos mais relevantes para enviar ao modelo.

**Pontos-chave para lembrar:**
- RAG = LLM + Dados externos ➜ respostas mais **precisas e atualizadas**.
- Aplicável em diversos contextos práticos (ex: atendimento ao cliente, busca em base interna).
- A qualidade da resposta depende diretamente dos dados que você fornece ao modelo.

> 🛠️ Aplicar RAG pode transformar suas aplicações com IA, tornando-as mais úteis, contextuais e confiáveis.

### Document Loaders – Carregando dados com Langchain

**Carregando PDFs**

In [1]:
# importando a classe necessária para manipular pdf
from langchain_community.document_loaders.pdf import PyPDFLoader

In [2]:
# aqui é o caminho onde ficará o arquivo
caminho = "arquivos_teste/contrato_aluguel_jose_nilton.pdf"

In [3]:
# fornecendo o pdf no loader
loader = PyPDFLoader(caminho)
# agora estamos carregando o pdf
documentos = loader.load()

In [4]:
# aqui a gente vê a quantidade de páginas do documento
len(documentos)

5

In [5]:
# aqui por exemplo estamos vendo o conteúdo da primeira página
documentos[0].page_content

'CONTRATO DE LOCAÇÃO RESIDENCIAL\nO LOCADOR e o LOCATÁRIO, qualificados abaixo (em conjunto denominados “Partes”,\ne, isoladamente, “Parte”), celebram este “ Instrumento Particular de Contrato de\nLocação” (“Contrato”), que será regido pelo disposto nas Leis federais nºs \xa08.245/1991\n(“Lei do Inquilinato”) e 10.406/2002 (“Código Civil”), e se comprometer a cumprir:\nCLÁUSULA PRIMEIRA: DA QUALIFICAÇÃO DAS PARTES\nLOCADOR: KAIO OLIVEIRA PEIXOTO, BRASILEIRO, SOLTEIRO, portador da cédula de\nidentidade R.G. nº 315622456 e CPF nº 021.667.755-60, residente e domiciliado na\nRua João Pessoa, nº 401, CEP 45745-000, bairro Centro, Ibicaraí-BAHIA;\nLOCATÁRIO:  JOSE  NILTON  PEREIRA  SILVA ,  BRASILEIRO,  SOLTEIRO,  COMERCIANTE,\nportador da cédula de identidade R.G. nº 15403176-33 e CPF nº 054.568.205-36,\nresidente e domiciliado na Rua Paraguaçu, nº 272, CEP 45745-000, bairro Centro,\nIbicaraí-BAHIA;\nCLÁUSULA SEGUNDA: OBJETO\nPor meio deste Contrato, o LOCADOR entrega ao LOCATÁRIO a posse e

In [6]:
# e aqui a gente pode ver alguns metadados
documentos[0].metadata

{'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf', 'page': 0}

Fazendo perguntas para o arquivo

In [7]:
# importando a chain necessária para interagir com o pdf
from langchain.chains.question_answering import load_qa_chain
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

# ao criar a chain, se colocarmos o verbose como True
# ele vai mostrar como trabalha debaixo dos panos
chain = load_qa_chain(llm=chat, chain_type="stuff", verbose=False)

In [8]:
pergunta = "fale de forma bem resumida em no máximo \
    10 palavras do q se trata esse documento?"

# agora vamos rodar a chain, veja q ela precisa de 2 inputs
# primeiro passamos o documento que queremos analisar
# e depois a pergunta
resposta = chain.invoke({'input_documents':documentos, 'question':pergunta})

resposta['output_text']

'Contrato de locação residencial entre Kaio e Jose.'

**Carregando CSVs**

Veja que o processo é bem parecido com o carregamento de PDFs.

A maior mudança é no nome do loader mesmo.

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [10]:
# aqui é o caminho onde ficará o arquivo
caminho_csv = "arquivos_teste/Top 1000 IMDB movies.csv"
loader = CSVLoader(caminho_csv)
documento_csv = loader.load()

In [11]:
# cada 'página' do documento seria uma linha do arquivo csv
len(documento_csv)

1000

In [12]:
# essa é a primeira linha
documento_csv[0].page_content

': 0\nMovie Name: The Shawshank Redemption\nYear of Release: (1994)\nWatch Time: 142 min\nMovie Rating: 9.3\nMeatscore of movie: 81\nVotes: 34,709\nGross: $28.34M\nDescription: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

In [13]:
# veja tbm q inclusive a criação da chain é igual à da leitura de pdf

chat = ChatOpenAI()

chain_csv = load_qa_chain(llm=chat, chain_type='stuff')

In [14]:
# a única diferença para o pdf é que forneceremos
# os documentos csv como input
pergunta = "dessa lista aí, cite o filme pior rankeado e qual o nome dele"

resposta = chain_csv.invoke({'input_documents':documento_csv[:50], 'question':pergunta})

resposta['output_text']

'Desculpe, não consigo responder a essa pergunta com base nas informações fornecidas.'

**YOUTUBE**

Para "assistir" vídeos do youtube deve-se fazer o processo abaixo.

Ele vai acessar o link e transcrever o audio através da api da OpenAI, com o método whisper.

Seria necessária a instalação de algumas bibliotecas ainda para completar essa solução, mas a base está aí.

In [15]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser

In [16]:
url = 'https://www.youtube.com/watch?v=Px8Iu2viG7I'
save_dir = 'arquivos_teste'
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser()
)

docs = loader.load()

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


[youtube] Extracting URL: https://www.youtube.com/watch?v=Px8Iu2viG7I
[youtube] Px8Iu2viG7I: Downloading webpage
[youtube] Px8Iu2viG7I: Downloading ios player API JSON
[youtube] Px8Iu2viG7I: Downloading mweb player API JSON
[youtube] Px8Iu2viG7I: Downloading player 73381ccc


         player = https://www.youtube.com/s/player/73381ccc/player_ias.vflset/en_US/base.js
         n = 1Rq63z-Csi5joMKSz ; player = https://www.youtube.com/s/player/73381ccc/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/73381ccc/player_ias.vflset/en_US/base.js
         n = Fy6og7FLVKv-yiIXg ; player = https://www.youtube.com/s/player/73381ccc/player_ias.vflset/en_US/base.js


[youtube] Px8Iu2viG7I: Downloading m3u8 information
[info] Px8Iu2viG7I: Downloading 1 format(s): 140
[download] arquivos_teste/AMIGO NOVO.m4a has already been downloaded
[download] 100% of    3.44MiB
[ExtractAudio] Not converting audio arquivos_teste/AMIGO NOVO.m4a; file is already in target format m4a


ImportError: pydub package not found, please install it with `pip install pydub`

**URL**

Agora vamos ler páginas de internet.

In [15]:
from langchain_community.document_loaders.web_base import WebBaseLoader

In [16]:
url = 'https://www.uol.com.br/esporte/futebol/ultimas-noticias/2025/03/31/rb-bragantino-ceara-brasileirao-serie-a-2025.htm'

loader = WebBaseLoader(url)
documentos_url = loader.load()

In [17]:
# 1 só pq é uma página só q eu passei
len(documentos_url)

1

In [18]:
# agora veja novamente q o processo de interagir com 
# o arquivo é igual às anteriores
chat = ChatOpenAI()

chain_url = load_qa_chain(llm=chat, chain_type='stuff')

In [19]:
pergunta = "faça um resumo dessa notícia com até 50 palavras"

resposta = chain_url.invoke({'input_documents':documentos_url, 'question':pergunta})

resposta['output_text']

'RB Bragantino empata com Ceará na estreia do Brasileirão Série A de 2025. Pedro Raul e Marllon marcaram para o Ceará, enquanto Laquintana e Eric Ramires fizeram os gols do Bragantino. Resultado de 2 a 2 deixou as equipes na sexta e sétima posição, respectivamente. Próximos jogos no final de semana.'

### Text Splitters – Dividindo texto em trechos

Agora q já sabemos carregar os documentos, precisamos saber como "quebrar" esse documento em pedaços menores para ser melhor consumido pelo modelo.

Essa é a segunda etapa do processo de RAG. As proximas etapas são embedar, guardar numa vector store e finalmente fazer o retrieval.

= = = = 

Aqui o objetivo é dividir o documento em trechos menores para fornecer ao LLM apenas pedaços relevantes da informação e assim ele processar. Isso garante mais agilidade e precisão no resultado final.

Mas como podemos manter a qualidade da informação se a dividimos em pedaços menores?

Vejamos esta frase de exemplo: "O novo carro da Fiat se chama Toro, tem 120 cavalos de potência e o preço sugerido é 135 mil reais."

Digamos q o split dessa frase ficaria da seguinte forma:

- "O novo carro da Fiat se"
- " chama Toro, tem 120 "
- "cavalos de potência e "
- "o preço sugerido é "
- "135 mil reais."

Aqui percebemos que os trechos separados individualmente não possuem valor. Sendo assim, seria de pouco valor para uma LLM, pois foi mal dividido.

Uma das técnicas para resolver isso é usar o parametro do overlapping. Esta técnica faz com que cada chunk tenha um pedaço do chunk anterior e do próximo. Dessa forma o LLM flui pela informação podendo fazer ligações.

Ficaria assim, por exemplo:

- "O novo carro da Fiat se"
- "da Fiat se chama Toro, tem 120 "
- "Toro, tem 120 cavalos de potência e "



In [20]:
# esse é o texto q vamos splittar
texto_grande = """
More and more organizations are turning to dashboards for monitoring performance and \
enabling data exploration. These user-friendly reporting tools offer a ton of \
advantages over older ways of doing things: they can dynamically update to display\
the latest information, link together multiple views of data, and often incorporate \
interactivity that lets users filter and zoom in on what they want to explore. 
"""

Existem formas diferentes de text splitter. Vamos explorar as mais utilizadas:

**CharacterTextSplitter**

In [21]:
from langchain_text_splitters import CharacterTextSplitter

In [22]:
# o tamanho do chunk (fatias) que vc deseja dividir
chunk_size = 50
# o tamanho da sobreposição. Geralmente um tamanho de 10 a 20% do chunk já funciona
chunk_overlap = 10

# criando o objeto q irá fazer o split
char_split = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator='' # o separator indica onde a divisão será feita
)

In [23]:
# vamos dar um exemplo de split com o abecedário
# o abecedario está disponivel importando a lib string
import string

In [24]:
# vamos criar nosso texto a ser dividido
# pegamos o abecedario e o replicamos 5 vezes
# veja q o tamanho do nosso texto ficou em 134 caracteres
texto = '-'.join(f'{string.ascii_lowercase}' for _ in range(5))
print(texto)
print(len(texto))

abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz
134


In [25]:
# agora fazendo o split do texto criado acima
# veja q o tamanho ficou em 4, sendo o chunksize de 50 
# e um total de 134 caracteres
splits = char_split.split_text(texto)
print(len(splits))
splits

4


['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvw',
 'nopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghi',
 '-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuv',
 'mnopqrstuvwxyz']

>> Veja acima como funciona o overlap. O segundo elemento da lista inicia com os 10 últimos caracteres da primeira lista, e assim sucessivamente. Ou seja, existe uma sobreposição entre o fim de um elemento da lista e inicio do próximo. A quantidade de caracteres que faz essa sobreposição é dada no chunk_overlap. E, claro, isso só não ocorre com o primeiro elemento, já q ele não tem antecessor.

**RecursiveCharacterTextSplitter**

Esse é o mais utilizado, ele permite o uso de vários separadores.

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
chunk_size = 50
chunk_overlap = 10

# criando o objeto q irá fazer o split
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [28]:
# vamos aproveitar o mesmo exemplo usado anteriormente
texto = '-'.join(f'{string.ascii_lowercase}' for _ in range(5))
len(texto)

134

In [29]:
char_split.split_text(texto)

['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvw',
 'nopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghi',
 '-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuv',
 'mnopqrstuvwxyz']

Agora utilizando o argumento separators

Com este argumento o splitter cria uma ordem de prioridade para quebrar os chunks. No exemplo abaixo, primeiro ele quebra por ponto, depois por espaço vazio e depois por qualquer lugar do texto.



In [30]:
# criando o objeto q irá fazer o split
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.', ' ', '']
)

# geralmente apenas uns poucos marcadores de split
# são necessários. São eles: ['\n\n', '\n', ' ', '']

Vamos ver um exemplo com um texto grande, com mais caracteres do que o abecedário.

In [31]:
char_split.split_text(texto_grande)

['More and more organizations are turning to',
 'to dashboards for monitoring performance and',
 'and enabling data exploration',
 '. These user-friendly reporting tools offer a ton',
 'a ton of advantages over older ways of doing',
 'of doing things: they can dynamically update to',
 'update to displaythe latest information, link',
 'link together multiple views of data, and often',
 'and often incorporate interactivity that lets',
 'that lets users filter and zoom in on what they',
 'what they want to explore',
 '.']

**TokenTextSplitter**

Permite o fatiamento em tokens em vez de quantidade de caracteres.

In [32]:
from langchain_text_splitters import TokenTextSplitter

In [33]:
chunk_size = 50
chunk_overlap = 10

token_split = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [34]:
token_split.split_text(texto)

['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijkl',
 'uvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz']

>Agora veja como fica a divisão por token. O abecedário agora foi dividido em apenas duas partes. Isso porque aquela primeira parte toda já é equivalente a 50 tokens. Essa forma de split facilita nosso gerenciamento das APIs da OpenAI e outras, que contabilizam o fluxo de dados em tokens.

**MarkdownHeaderTextSplitter**

Podemos fazer o split de textos em Markdown. Existem vários marcadores nesse tipo de texto que podemos indicar para o split. Dessa forma fica mais fácil identificarmos quando trata-se de um texto principal (#), subtitulo (##), e etc.

In [35]:
markdown_text = """# Exemplo de titulo em markdown
## Aqui é um subtitulo
texto q fica dentro desse subtitulo
### Aqui outra hierarquia de header
Mais texto livre q fica dentro desta hierarquia
**e vamos de mais exemplos**
"""

In [36]:
# importando a biblioteca necessária
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [37]:
# primeiro criamos uma lista de tuplas
# onde terão os marcadores que farão o split

header_to_split_on = [
    ('#', 'Header 1'),
    ('##', 'Header 2'),
    ('###', 'Header 3'),
]

# criando o splitter
md_split = MarkdownHeaderTextSplitter(
    headers_to_split_on=header_to_split_on
)

In [38]:
# fazendo o split
splits = md_split.split_text(markdown_text)

In [39]:
splits

[Document(page_content='texto q fica dentro desse subtitulo', metadata={'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo'}),
 Document(page_content='Mais texto livre q fica dentro desta hierarquia\n**e vamos de mais exemplos**', metadata={'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo', 'Header 3': 'Aqui outra hierarquia de header'})]

>Veja como fica o output. Ele traz as hierarquias de título\subtitulo em metadata e traz seu conteúdo na variável page_content.

**Split de documentos**

Vamos fazer o split de um documento, em vez de textos simples como fizemos anteriormente.

In [40]:
# importando a biblioteca q carregará e nos ajudará a trabalhar com doc
from langchain_community.document_loaders import PyPDFLoader
# importando um splitter recursivo
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [41]:
chunk_size = 50
chunk_overlap = 10

# criando o objeto q irá fazer o split
# até aqui é identico ao q já fizemos anteriormente
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [42]:
# caminho para o arquivo
caminho = 'arquivos_teste/contrato_aluguel_jose_nilton.pdf'

In [43]:
# carregando o arquivo
loader = PyPDFLoader(caminho)
docs = loader.load()

# veja que o tamanho é apenas 5, q é a quantidade de páginas
len(docs)

5

In [44]:
# agora vejamos depois de fazer o split
# fique atento que aqui o método é diferente, sendo split_documents
# em vez de split_text
doc_split = char_split.split_documents(docs)
len(doc_split)

227

### Embeddings - Transformando texto em vetores

Embeddings criam uma representação vetorial de um pedaço de texto. Ele transforma letras em números. Isso é útil, pois é assim que o modelo vai "pensar" de forma mais eficiente. Dessa forma ele pode buscar proximidades semanticas, etc. Os cálculos vetoriais é quem dão a "consciencia" e "discernimento" para o modelo.

O Langchain fornece uma classe de embedding que interage com os modelos de embedding fornecidos no mercado, ou seja, existem ebeddings da OpenAI, HuggingFace, Claude etc. O Langchain fornece uma interface padrão para todas elas.

In [45]:
from langchain_openai import OpenAIEmbeddings

In [46]:
# o modelo default para embedding é esse q está no argumento da chamada
# existem outros. Visite o site da OpenAI para detalhes
# vamos instanciar esse embedding da OpenAI
embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002')

**Embedding de Documentos**

In [47]:
# vamos criar nosso modelo 
# veja que ele possui uma lista, onde ficarão os documentos
# inserimos 3 documentos, q no momento são apenas frases
embeddings = embedding_model.embed_documents(
    [
        'Eu amo frutas!',
        'Adoro comer maçã e morando no café da manhã',
        'Eu não gostei dessa piada.'
    ]
)

In [48]:
# temos 3 documentos
len(embeddings)

3

In [49]:
# vamos como ficou vetorizada a primeira frase
# aqui vemos os 10 primeiros vetores
embeddings[0][:10]

[-0.006634879172388902,
 -0.01640579910410335,
 0.008534951452491854,
 -0.018041460513549958,
 0.00036875386927730914,
 0.012144472901045993,
 0.004282848602894743,
 -0.01683623592497978,
 0.02096843238562586,
 -0.018189037894204337]

In [50]:
# vamos fazer um for e analisar cada vetor
# primeiro vemos q o tamanho dos vetores é o mesmo (1536)
# isso acontece por causa do modelo escolhido da OpenAI, na hora de instanciar
# depois pegamos o valor máximo e o mínimo de cada lista de vetores
for emb in embeddings:
    print(len(emb), max(emb), min(emb))

1536 0.22313854927024132 -0.6505255940753462
1536 0.24039689057922564 -0.6608969665847052
1536 0.22472701344464535 -0.6345005455745475


In [51]:
# agora vamos fazer uma multiplicação vetorial utilizando o numpy
# para identificarmos as semelhanças semanticas entre os vetores
# é através desses cálculo que o modelo 'pensa' e sabe das semelhanças
# entre as frases. Quanto mais próximo de 1, mais semelhantes são as frases

import numpy as np

In [52]:
# multiplicando a primeira e a segunda frase
# veja o número que obtemos
# quanto mais alto significa que as frases tem semânticas próximas
np.dot(embeddings[0], embeddings[1])

0.8841020582306888

In [53]:
# agora veja a comparação da frase 1 com a frase 3, o número é menor
np.dot(embeddings[0], embeddings[2])

0.7906537384786307

In [54]:
# mesma coisa entre as frases 2 e 3
np.dot(embeddings[1], embeddings[2])

0.7688946865216448

In [55]:
# vamos ver o conjunto todo, multiplicando todos vetores entre eles
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        print(round(np.dot(embeddings[i], embeddings[j]), 3), end=' | ')
    print()

1.0 | 0.884 | 0.791 | 
0.884 | 1.0 | 0.769 | 
0.791 | 0.769 | 1.0 | 


**Embedding Query**

Precisamos tbm fazer o embedding da pergunta do usuário ao modelo.

In [56]:
# agora vamos utilizar o método emb_query() do modelo instanciado
pergunta = "quais os alimentos mais saudáveis"
emb_query = embedding_model.embed_query(pergunta)
emb_query[:10] # vendo os 10 primeiros vetores dessa pergunta

[0.011823595855325887,
 -0.0015175153225819675,
 0.003353206044200978,
 -0.004639096331514772,
 0.003132296805952957,
 0.004395107058349887,
 -0.00871108243536731,
 -0.0341848975114945,
 -0.017171579505757185,
 0.0036796243461127056]

In [57]:
# agora comparando com as frases criadas antes
# veja como a pergunta é próxima das frase 1 por exemplo
np.dot(emb_query, embeddings[1])

0.8125198875109063

In [58]:
# e veja com ela é mais distante da frase 3
np.dot(emb_query, embeddings[2])

0.7236170898533685

**Embedding com Huggingface**

In [59]:
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings

In [60]:
model = 'all-MiniLM-L6-v2'
emb_model = HuggingFaceBgeEmbeddings(model_name=model)

/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [61]:
embeddings = emb_model.embed_documents(
    [
    'Eu gosto de manter uma alimentação saudável.',
    'Adoro comer maçã e morango no café da manhã.',
    'A crise financeira de 2008 abalou o mundo.']
)

In [62]:
# vamos ver o conjunto todo, multiplicando todos vetores entre eles
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        print(round(np.dot(embeddings[i], embeddings[j]), 3), end=' | ')
    print()

1.0 | 0.432 | 0.419 | 
0.432 | 1.0 | 0.313 | 
0.419 | 0.313 | 1.0 | 


### VectorStores

Uma VectorStore faz o armazenamento de vetores e realiza a busca desses vetores. Depois de fazermos o embedding, como explicado na aula anterior, temos que guardá-los num lugar otimizado para trabalharmos com os vetores.

Temos várias soluções de VectorStores e aqui vamos estudar duas das mais utilizadas: Chroma e FAISS

**Chroma VectorStore**

In [63]:
# vamos fazer o processo desde o início
# ou seja, vamos carregar os documentos, splittar, fazer o embedding
# e guardar na VectorStore para retrieval

# importando as libs para carregar os docs
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [64]:
# carregando os documentos
caminho = 'arquivos_teste/contrato_aluguel_jose_nilton.pdf'
loader = PyPDFLoader(caminho)
docs = loader.load()

In [65]:
# agora vamos para o text splitting
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(docs)

In [66]:
#Criando a VectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [67]:
embeddings_model = OpenAIEmbeddings()

In [68]:
# diretorio onde a base de dados será salva
diretorio = "files/chroma_vectorstore"

In [69]:
# criando a vector store
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [70]:
# aqui a gente vê quandos documentos existem dentro da vector store
print(vector_store._collection.count())

205


In [71]:
# aqui agora vamos carregar uma vector store já existente
# veja q não utilizamos o método .from_documents()
# utilizamos somente a classe Chroma
# fornecemos apenas o modelo de embedding e o diretório
# onde a vector store está localizada
vector_store = Chroma(
    embedding_function=embeddings_model,
    persist_directory=diretorio
)

**Retrieval**

Fazendo perguntas para os documentos

In [72]:
# aqui vamos ver como o vector store procura pela similidaridade
# vamos fazer uma pergunta e ver quais trechos do documento ele retorna
pergunta = 'qual é o valor do aluguel?'

# o argumento k é a quantidade de documentos q ele vai pesquisar
docs = vector_store.similarity_search(pergunta, k=5)
len(docs)

5

In [73]:
# e agora veja o conteúdo de cada documento
# perguntamos sobre valor do aluguel
# e ele buscou os trechos q estão mais relacionados à pergunta
for doc in docs:
    print(doc.page_content)
    print("=====", doc.metadata, '\n')

Pagará o LOCATÁRIO ao LOCADOR o valor mensal de R$ 350,00, a título de aluguel. Os
aluguéis serão pagos todo dia 05 de cada mês, efetivando-se por depósito bancário no
Banco do Brasil, Agência 3028-7, Conta Corrente 31872-8, em nome de Kaio Oliveira
===== {'page': 0, 'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf'} 

Pagará o LOCATÁRIO ao LOCADOR o valor mensal de R$ 350,00, a título de aluguel. Os
aluguéis serão pagos todo dia 05 de cada mês, efetivando-se por depósito bancário no
Banco do Brasil, Agência 3028-7, Conta Corrente 31872-8, em nome de Kaio Oliveira
===== {'page': 0, 'source': 'contrato_aluguel_jose_nilton.pdf'} 

Pagará o LOCATÁRIO ao LOCADOR o valor mensal de R$ 350,00, a título de aluguel. Os
aluguéis serão pagos todo dia 05 de cada mês, efetivando-se por depósito bancário no
Banco do Brasil, Agência 3028-7, Conta Corrente 31872-8, em nome de Kaio Oliveira
===== {'page': 0, 'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf'} 

Pagará o LOCATÁRIO ao 

**FAISS VectorStore**

In [74]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [75]:
# carregando os documentos
caminho = 'arquivos_teste/contrato_aluguel_jose_nilton.pdf'
loader = PyPDFLoader(caminho)
docs = loader.load()

# agora vamos para o text splitting
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(docs)

In [76]:
# criando um modelo de embedding
embeddings_model = OpenAIEmbeddings()

In [80]:
# criando a vector store com Faiss
from langchain_community.vectorstores.faiss import FAISS

# agora vamos criar a vectorstore com FAISS
# é bem parecido com o Chroma, mas sem o diretorio
# FAISS não usa 'persist_directory' porque não tem persistência nativa 
vector_store_faiss = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model,
)

In [81]:
# agora fazendo o retrieval com essa nova base FAISS
# aqui vamos ver como o vector store procura pela similidaridade
# vamos fazer uma pergunta e ver quais trechos do documento ele retorna
pergunta = 'qual é o valor do aluguel?'

# o argumento k é a quantidade de documentos q ele vai pesquisar
docs = vector_store_faiss.similarity_search(pergunta, k=5)
len(docs)

5

In [82]:
# e agora veja o conteúdo de cada documento
# perguntamos sobre valor do aluguel
# e ele buscou os trechos q estão mais relacionados à pergunta
for doc in docs:
    print(doc.page_content)
    print("=====", doc.metadata, '\n')

Pagará o LOCATÁRIO ao LOCADOR o valor mensal de R$ 350,00, a título de aluguel. Os
aluguéis serão pagos todo dia 05 de cada mês, efetivando-se por depósito bancário no
Banco do Brasil, Agência 3028-7, Conta Corrente 31872-8, em nome de Kaio Oliveira
===== {'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf', 'page': 0} 

O  atraso  no  pagamento  do  aluguel  e/ou  dos  encargos  da  locação  implicará  a
incidência de multa de 2,00% (dois por cento) sobre o valor total de débito e juros de
1,00% (um por cento) ao mês.
CLÁUSULA SÉTIMA: USO E DESTINAÇÃO DA LOCAÇÃO
===== {'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf', 'page': 1} 

Peixoto.
O aluguel pactuado acima sofrerá reajustes anuais com base na variação do Índice
Geral de Preços divulgado pela Fundação Getúlio Vargas (IGP-FGV) ou outro índice que
porventura venha a substituí-lo.
===== {'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf', 'page': 0} 

(três)  vezes  o  valor  do  aluguel  vigente  à  ép

**Salvando o bd do FAISS**

Observação importante

- Chroma: sempre precisa apontar para uma pasta (`persist_directory`) se você quiser que ele salve os dados no disco. Sem isso, ele funciona só na memória durante a execução, mas para manter o banco de dados entre execuções, é necessário definir a pasta.

- FAISS: não precisa apontar pasta na criação, porque por padrão ele trabalha só na memória RAM. Mas se quiser salvar o banco e reutilizar depois, você precisa usar `.save_local()` e `.load_local()` manualmente.

In [84]:
# salvando...
vector_store_faiss.save_local('arquivos_teste/faiss_bd')

In [85]:
# importando o bd faiss
# o argumento dangerous deserialization é usado 
# como True quando vc confia nos dados de origem,
# pois ele pode deserializar código malicioso
# já q eu criei o banco e sei a origem, então deixo como True
vector_store_faiss = FAISS.load_local(
    'arquivos_teste/faiss_bd',
    embeddings=embeddings_model,
    allow_dangerous_deserialization=True
)

🧩 **Resumo Vector Store: Chroma vs FAISS**

**Essência em 1 parágrafo:**
Tanto Chroma quanto FAISS são vector stores que armazenam e fazem buscas por embeddings, mas a principal diferença está na facilidade de uso e integração. Chroma é pensado para ser plug-and-play, com armazenamento local por padrão e até persistência nativa, ótimo para protótipos e projetos de médio porte. FAISS é uma biblioteca da Meta, mais "pura", focada em alta performance e escala, muito usada para produção em grandes volumes de dados, mas exige mais configuração e não tem persistência embutida (você salva manualmente).

**Principais pontos:**

| Característica                  | Chroma                                | FAISS                                   |
|----------------------------------|----------------------------------------|------------------------------------------|
| 📦 Instalação e uso              | Super fácil, abstraído, integrado      | Mais manual, requer mais configuração    |
| 💾 Persistência (salvar dados)    | Sim, nativa (`persist_directory`)      | Não nativa, precisa serializar manualmente |
| 🚀 Performance                   | Boa para médias bases                 | Muito rápido e otimizado para grandes bases |
| 🔗 Integração com LangChain      | Muito fluida, oficial                 | Também integra bem, mas demanda mais setup |
| 🔥 Escalabilidade                | Limitada, para protótipos/projetos médios | Alta escala, ideal para milhões de vetores |
| 🌐 Multi-idioma / Multi-device   | Suporte mais limitado                 | Suporte para CPU/GPU e multi-threading avançado |

**Observação importante sobre FAISS:**

> FAISS não usa 'persist_directory' porque não tem persistência nativa; ele guarda os dados na memória durante a execução, e para manter os dados depois de fechar o programa, é preciso salvar e carregar manualmente o índice.

**Quando usar cada um:**

- **Use Chroma se:**
  - Quer prototipar rápido.
  - Seu projeto tem até algumas dezenas ou centenas de milhares de vetores.
  - Você quer facilidade de persistência local.

- **Use FAISS se:**
  - Precisa de alta performance em bases grandes (milhões de vetores).
  - Vai rodar em produção com alta demanda.
  - Está confortável com setups mais avançados e salvar/restaurar índices manualmente.

---

🧭 **Outros Vector Stores para o dia a dia**

**Essência em 1 parágrafo:**
Além de Chroma e FAISS, outros vector stores aparecem bastante na prática, principalmente quando se busca escalabilidade profissional, integração com nuvem ou recursos avançados de busca híbrida (texto + vetores). Pinecone, Weaviate, Milvus e Redis Vector são ótimas opções dependendo do cenário.

**Principais opções:**

- **Pinecone:**  
  - SaaS dedicado e fácil de usar.
  - Escala automática, alta disponibilidade.
  - Integra bem com LangChain.
  - Ideal para produção sem dor de cabeça com infraestrutura.

- **Weaviate:**  
  - Open-source com busca híbrida (texto + vetores).
  - Permite filtros avançados por metadados.
  - Escalável e pronto para produção.

- **Milvus:**  
  - Open-source, alto desempenho para bilhões de vetores.
  - Suporte forte para GPU.
  - Indicado para Big Data e aplicações massivas.

- **Redis Vector:**  
  - Usa Redis para armazenar vetores.
  - Boa opção se você já usa Redis na stack.
  - Performance sólida e suporte a filtros por metadados.

**Resumo de escolha:**

- **Protótipo/local:** Chroma
- **Projeto pequeno, on-premise:** FAISS
- **Produção escalável:** Pinecone ou Weaviate
- **Big Data + GPU:** Milvus
- **Stack Redis:** Redis Vector

### Retrieval – Encontrando trechos relevantes

É a busca por trechos semelhantes à pergunta que o usuário fez.

Já fizemos isso acima na aula de vector sotres, e agora vamos ver de forma mais detalhada.


In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

Aqui vamos fazer um loading com 3 arquivos. Faremos um laço for para carregar os 3 arquivos pdf.

Mas veja que um desses arquivos está duplicado. Isso foi de forma intencional para vermos como os diferentes tipos de retrieval tratam um mesmo documento.

In [2]:
caminhos = [
    'arquivos_teste/contrato_aluguel_jose_nilton.pdf',
    'arquivos_teste/vantagem_pdf.pdf',
    'arquivos_teste/vantagem_pdf.pdf'
]

paginas = []

for doc in caminhos:
    loader = PyPDFLoader(doc)
    paginas.extend(loader.load()) # adicionando cada arquivo carregado na lista

# criando o splitter para dividir os documentos
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

# agora splitando os documentos
documents = recur_split.split_documents(paginas)

**Modificando metadata**

In [3]:
# sabemos que todo documento traz metadados
documents[0].metadata

{'source': 'arquivos_teste/contrato_aluguel_jose_nilton.pdf', 'page': 0}

In [4]:
# mas esses dados também podem ser editados
# e podemos também adicionar novos metadados
# vamos dar aqui um exemplo e modificar algumas coisas
# eu quero por exemplo tirar o nome arquivos_teste do nome do arquivo
# e adicionar um novo atributo, q é o id do documento, q terá o valor do índice
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos_teste/', '')
    doc.metadata['doc_id'] = i

In [5]:
# agora verificando depois da modificação
# veja como ficaram os novos atributos
documents[0].metadata

{'source': 'contrato_aluguel_jose_nilton.pdf', 'page': 0, 'doc_id': 0}

**Criando a VectorStore**

Vamos criar essa base de dados para interagir com os documentos q splitamos acima.

In [6]:
# primeiro criamos o embedding
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embedding_model = OpenAIEmbeddings()

In [7]:
# agora criando um banco com o chroma
diretorio = "arquivos_teste/chroma_retrieval"

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=diretorio
)

**Semantic Search**

Esse é o tipo de retrieval mais simples que existe.

In [8]:
pergunta = "quais são as vantagens de usar um arquivo pdf?"

# vamos pedir para ele trazer os 3 trechos mais relevantes
docs = vector_db.similarity_search(pergunta, k=3)

# agora vamos ver quais foram esses trechos q ele encontrou
for doc in docs:
    print(doc.page_content)
    print(f"====={doc.metadata}\n\n")

Vantagens do PDF  
 
 Pequenho tamanho do arquivo: os arquivos possuem uma compactação 
aceitável (ex.: Arquivos  de Word com 1Mb após a conversão para PDF 
chegam a ficar com 100 Kb de tamanho, 10% do  original);  
 Os arquiv os PDF são compactos e totalmente pesquisáveis, podendo ser 
acessados a qualquer momento com o Adobe Reader.  
 Não apresentam problemas de fontes e/ou formatação de arquivos;  
 Os documentos PDF podem ter direitos especiais de acesso e podem ser
====={'doc_id': 25, 'page': 1, 'source': 'vantagem_pdf.pdf'}


Vantagens do PDF  
 
 Pequenho tamanho do arquivo: os arquivos possuem uma compactação 
aceitável (ex.: Arquivos  de Word com 1Mb após a conversão para PDF 
chegam a ficar com 100 Kb de tamanho, 10% do  original);  
 Os arquiv os PDF são compactos e totalmente pesquisáveis, podendo ser 
acessados a qualquer momento com o Adobe Reader.  
 Não apresentam problemas de fontes e/ou formatação de arquivos;  
 Os documentos PDF podem ter direitos especiais

Veja que eu dupliquei o arquivo sobre pdf de propósito. Note que o modelo retornou 2 trechos repetidos, q são os 2 primeiros. Veja pelo doc_id que tratam-se de dois documentos diferentes, mas com conteúdo igual.

O modelo não conseguiu avaliar que o segundo trecho era exatamente igual ao primeiro e não adiciona nenhuma informação nova. Essa é uma das limitações desse tipo de retrieval.

Ele não leva em conta o metadado e outros parametros, tornando a busca menos eficiente.

**MMR (Max Marginal Relevance)**

- Traz resultados diversos, **evita repetição** entre documentos.
- Combina **relevância** com **diversidade** dos documentos retornados.
- Útil para não receber vários trechos muito parecidos na resposta.

Principais argumentos:
- **query:** sua pergunta em texto.
- **k:** total de documentos que você quer receber.
- **fetch_k:** número de documentos candidatos que ele vai considerar antes de escolher os melhores (default: maior que `k`).
- **lambda_mult:** equilíbrio entre relevância e diversidade (de 0 a 1).
  - 1 = mais foco na relevância.
  - 0 = mais foco na diversidade.

Vamos aqui fazer um teste fazendo a mesma pergunta q fizemos no similarity search.

Mas note como ele retorna uma resposta melhor.

In [9]:
pergunta = "quais são as vantagens de usar um arquivo pdf?"

# vamos pedir para ele trazer os 10 trechos mais similares com fetch_k
# e no argumento k ele vai trazer os 3 mais relevantes
docs = vector_db.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)

# agora vamos ver quais foram esses trechos q ele encontrou
for doc in docs:
    print(doc.page_content)
    print(f"====={doc.metadata}\n\n")

Vantagens do PDF  
 
 Pequenho tamanho do arquivo: os arquivos possuem uma compactação 
aceitável (ex.: Arquivos  de Word com 1Mb após a conversão para PDF 
chegam a ficar com 100 Kb de tamanho, 10% do  original);  
 Os arquiv os PDF são compactos e totalmente pesquisáveis, podendo ser 
acessados a qualquer momento com o Adobe Reader.  
 Não apresentam problemas de fontes e/ou formatação de arquivos;  
 Os documentos PDF podem ter direitos especiais de acesso e podem ser
====={'doc_id': 25, 'page': 1, 'source': 'vantagem_pdf.pdf'}


O que é Adobe PDF?  
 
O PDF (Portable Document Format) é um formato de arquivo  
desenvolvido pela Adobe Systems para representar  documentos de maneira 
independente do aplicativo,  hardware, e sistema operacional usados para 
criá-los. Um  arquivo PDF pode descrever documentos que contenham  texto, 
gráficos e imagens num formato independente de  dispositivo e resolução.  
Sua principal característica c onsiste em representar um documento
====={'doc_

🎯 Relação entre fetch_k e k

- **fetch_k** é o número de documentos candidatos que o método vai considerar primeiro.
- **k** é o número final de documentos que você realmente quer como resposta.

🔎 **Funcionamento:**
1. Primeiro, ele pega os **fetch_k** documentos mais relevantes.
2. Depois, ele filtra entre esses candidatos para escolher os **k** resultados finais, equilibrando **relevância e diversidade** (controlado pelo `lambda_mult`).

Exemplo:
```python
k = 5          # Quero 5 documentos no final
fetch_k = 20   # Ele vai considerar 20 candidatos antes de escolher

**Filtragem**

O filter é um argumento dentro do similarity_search que limita os resultados da busca aos documentos que correspondem aos metadados especificados.

- O **filter** permite **filtrar** os documentos recuperados com base nos metadados que você salvou junto com os documentos.
- Serve para refinar a busca, trazendo só documentos que atendam a certos critérios.

Exemplo:

Se seus documentos têm metadado `{"categoria": "contrato"}`, você pode buscar só nessa categoria:
```python
docs = vector_store.similarity_search(
    query=pergunta,
    k=5,
    filter={"categoria": "contrato"}
)

Vamos ver abaixo com o nosso exemplo real.

In [10]:
pergunta = "quais são as vantagens de usar um arquivo pdf?"

docs = vector_db.similarity_search(
    pergunta, 
    k=3,
    filter={'doc_id':21})

# agora veja q ele só buscou dentro do doc_id = 0
for doc in docs:
    print(doc.page_content)
    print(f"====={doc.metadata}\n\n")

O que é Adobe PDF?  
 
O PDF (Portable Document Format) é um formato de arquivo  
desenvolvido pela Adobe Systems para representar  documentos de maneira 
independente do aplicativo,  hardware, e sistema operacional usados para 
criá-los. Um  arquivo PDF pode descrever documentos que contenham  texto, 
gráficos e imagens num formato independente de  dispositivo e resolução.  
Sua principal característica c onsiste em representar um documento
====={'doc_id': 21, 'page': 0, 'source': 'vantagem_pdf.pdf'}


O que é Adobe PDF?  
 
O PDF (Portable Document Format) é um formato de arquivo  
desenvolvido pela Adobe Systems para representar  documentos de maneira 
independente do aplicativo,  hardware, e sistema operacional usados para 
criá-los. Um  arquivo PDF pode descrever documentos que contenham  texto, 
gráficos e imagens num formato independente de  dispositivo e resolução.  
Sua principal característica c onsiste em representar um documento
====={'doc_id': 21, 'page': 0, 'source': 'van

**LLM Aided Retrieval**

- LLM-Aided Retrieval usa o próprio modelo de linguagem (LLM) para **ajudar na recuperação de documentos**, melhorando a qualidade das buscas.
- Em vez de só usar vetores para buscar, o LLM pode:
  - Reescrever ou expandir a pergunta antes de buscar.
  - Filtrar ou rankear melhor os resultados recuperados.
  - Fazer busca multi-turno (exemplo: perguntar ao LLM como refinar a consulta).

Por que usar?
- Ajuda quando a pergunta do usuário não é tão clara.
- Aumenta a precisão em buscas mais complexas ou ambíguas.

Resumindo, em vez de você escrever manualmente filtros ou ajustes na busca, o modelo "pensa por você" e traduz a pergunta em uma query estruturada.

In [11]:
# vamos importar as libs q precisamos
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [12]:
documents[0].metadata

{'source': 'contrato_aluguel_jose_nilton.pdf', 'page': 0, 'doc_id': 0}

In [13]:
# aqui vamos apresentar para o modelo em quais atributos
# ele deve prestar atenção e uma breve descrição de cada um
metadata_info = [
    AttributeInfo(
        name='source',
        description='é o nome da apostila, onde o texto original foi retirado. \
            Deve ter o valor de: contrato_aluguel_jose_nilton.pdf',
            type='string'
    ),
    AttributeInfo(
        name='page',
        description='a página da apostila de onde o texto se origina.',
            type='integer'
    )
]

In [14]:
# fornecendo a descrição do documento como um todo
content_description = 'apostila de cursos'
llm = OpenAI()

# e aqui criamos o nosso aided retriever
retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_db,
    content_description,
    metadata_info,
    verbose=True
)

In [16]:
pergunta = "qual a data de assinatura do contrato?"

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f"====={doc.metadata}\n\n")

As  partes  elegem  o  foro  da  comarca  de  Ibicaraí/BA ,  para  dirimir  quaisquer
controvérsias oriundas do presente instrumento.
As partes declaram que tiveram acesso a este contrato, leram, compreenderam e que
estão de acordo com todas as suas cláusulas, prometendo ainda a cumprirem e
fazerem que se cumpra o presente Contrato de Locação.
Por estarem assim justos e contratados, firmam o presente instrumento, em duas vias
de igual teor.
Ibicaraí, 01 de Agosto de 2023.
====={'doc_id': 17, 'page': 3, 'source': 'contrato_aluguel_jose_nilton.pdf'}


As  partes  elegem  o  foro  da  comarca  de  Ibicaraí/BA ,  para  dirimir  quaisquer
controvérsias oriundas do presente instrumento.
As partes declaram que tiveram acesso a este contrato, leram, compreenderam e que
estão de acordo com todas as suas cláusulas, prometendo ainda a cumprirem e
fazerem que se cumpra o presente Contrato de Locação.
Por estarem assim justos e contratados, firmam o presente instrumento, em duas vias
de igual teor.

### RAG – Conversando com os seus dados

In [17]:
# primeiro vamos fazer todo o processo do zero
# importando as bibliotecas
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
# criando o passo a passo

# indicando o caminho dos meus arquivos
caminhos = [
    'arquivos_teste/contrato_aluguel_jose_nilton.pdf',
    'arquivos_teste/vantagem_pdf.pdf',
]

# fazendo o load das páginas usando o pdf loader
paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

# dividindo os documentos para ele ser melhor consumido pelo modelo
# criando o splitter
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "", " "]
)

# aqui a gente executa o split
documents = recur_split.split_documents(paginas)

# modificando alguns metadados para facilitar a interação (opcional)
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos_teste/', '')
    doc.metadata['doc_id'] = i

In [19]:
# criando a vector db

# indicando onde ficará o banco de dados
diretorio_db = "arquivos_teste/chroma_retrieval_db"

# criando o embedder
embedding_model = OpenAIEmbeddings()

# juntando tudo e criando o vector db com o Chroma
vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=diretorio_db
)


2025-04-12 13:12:26,805 - INFO - Backing off send_request(...) for 1.0s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Read timed out. (read timeout=15))


**Criando a Estrutura da Conversa**

In [20]:
# para criar a estrutura de chat precisamos instanciar o chat open ai
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [25]:
# agora precisamos criar uma chain para unir e interagir com toda essa estrutura
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vector_db.as_retriever(search_type='mmr'),
)

In [28]:
# agora vamos interagir!
pergunta = "quem é o autor do artigo q fala sobre pdf?"

chat_chain.invoke({'query': pergunta})

{'query': 'quem é o autor do artigo q fala sobre pdf?',
 'result': 'O autor do artigo sobre PDF é a Secretaria de Tecnologia da Informação do TRT da 4ª Região.'}

**Modificando o prompt da chain**

Já interagimos com o modelo, agora podemos moldá-lo para que ele siga um comportamento específico ou faça as coisas da maneira que queremos.

Aqui entra a engenharia de prompt.

In [44]:
from langchain.prompts import PromptTemplate

In [49]:
# ao criar o prompt precismos usar as palavras chave context e question
# o context vai armazenar todos os documentos q fornecemos ao modelo
# question representa a pergunta
chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornecido para responder a pergunta ao final.
Fale em tom de humor e descontração, pode manter a informalidade.
Se você não sabe a resposta, apenas diga que não sabe e não tente inventar resposta.
Utilize três frases no máximo, mantenha a resposta concisa.


Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

In [50]:
# agora depois de criado o prompt
# precisamos criar a chain novamente para incluí-lo
# veja q usamos o argumento chain type kwargs
# e incluimos o novo prompt num dicionário
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vector_db.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt':chain_prompt},
    return_source_documents=True 
)

# o argumento return_source_documents retorna as fontes pesquisadas
# usamos aqui só para 'provar' q ele está usando o novo prompt

In [51]:
# agora vamos ver como ele se comporta depois do prompt
# vamos guardar o retorno na variável 'resposta'
# esse retorno é um dicionário
pergunta = "o q é adobe pdf?"

resposta = chat_chain.invoke({'query': pergunta})

In [52]:
# vamos ver a chave result, q é a resposta do modelo
# veja q ele retornou a resposta em 3 frases, conforme pedimos no prompt
# veja q ele também mudou o tom para um pouco de humor
print(resposta['result'])

O Adobe PDF é tipo o super-herói dos arquivos, salva você de qualquer confusão com formatos diferentes.  
É tipo um uniforme que faz qualquer documento ficar bonitão e acessível em qualquer dispositivo.  
Pra resumir, é tipo uma varinha mágica que transforma seus arquivos em algo indestrutível e prontos pra qualquer situação.


In [54]:
# agora tbm podemos acessar a chave source_documents
# ele traz todos os arquivos q o modelo utilizou para gerar a resposta
resposta['source_documents']

[Document(page_content='O que é Adobe PDF?  \n \nO PDF (Portable Document Format) é um formato de arquivo  \ndesenvolvido pela Adobe Systems para representar  documentos de maneira \nindependente do aplicativo,  hardware, e sistema operacional usados para \ncriá-los. Um  arquivo PDF pode descrever documentos que contenham  texto, \ngráficos e imagens num formato independente de  dispositivo e resolução.  \nSua principal característica c onsiste em representar um documento', metadata={'doc_id': 28, 'page': 0, 'source': 'vantagem_pdf.pdf'}),
 Document(page_content='essenciais da tecnologia PostScript, que é uma espécie de linguagem usada \npara  const ruir páginas para os mais diversos fins.  \nEm geral, é possível transformar qualquer arquivo que possa ser impre sso \nem arquivos PDF.   \n \n \nSecretaria de Tecnologia da Informação do TRT da 4ª Região', metadata={'doc_id': 31, 'page': 0, 'source': 'vantagem_pdf.pdf'}),
 Document(page_content='assinados digitalmente , ou seja, i mpedem 

#### Outros tipos de chains

🔄 chain_type='stuff' — o que é e como funciona

- O `chain_type='stuff'` define como os documentos recuperados vão ser passados para o modelo LLM gerar a resposta.
- No modo `"stuff"`, **todos os documentos recuperados são "empilhados" (stuffed) em um único prompt**, e esse prompt é enviado direto ao modelo.

Exemplo prático:
Se você recuperou 5 documentos com `k=5`, o LangChain junta todos os textos desses documentos num só prompt e diz:
> "Com base nesses textos, responda a pergunta..."

**✅ Vantagem:** simples e direto, ótimo para poucos documentos pequenos.  
**⚠️ Limite:** se os textos forem muito grandes, pode ultrapassar o limite de tokens do modelo.

---

🔁 Outros `chain_type` disponíveis:

1. `"map_reduce"`  
- Divide os documentos em partes.
- Roda o LLM em cada parte separadamente (**map**) e depois junta os resultados (**reduce**).
- Melhor para textos longos.

2. `"refine"`  
- Cria uma resposta inicial com o primeiro documento e vai **refinando** essa resposta com os próximos.
- Mais lento, mas pode melhorar a coerência.

3. `"map_rerank"`  
- Avalia cada documento individualmente e escolhe a melhor resposta com uma nota de relevância.
- Útil quando você quer **qualidade acima de quantidade**.

---

✅ Conclusão curta:
> `stuff` junta todos os docs e envia de uma vez ao LLM — simples e eficiente para entradas pequenas.  
> Para documentos longos, prefira `map_reduce` ou `refine`.



**Stuff**

Quando a gente une nosso prompt com o contexto, existem várias formas de fazer isso.

A forma mais simples e mais utilizada é a stuff. Esse também é o valor default.

In [58]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vector_db.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = "o q é adobe pdf?"

resposta = chat_chain.invoke({'query': pergunta})

print(resposta['result'], end='\n')

O Adobe PDF (Portable Document Format) é um formato de arquivo desenvolvido pela Adobe Systems para representar documentos de maneira independente do aplicativo, hardware e sistema operacional usados para criá-los. Ele permite a representação de documentos contendo texto, gráficos e imagens de forma independente de dispositivo e resolução. É comumente utilizado para compartilhar documentos de forma segura e preservar a formatação original.


**Map Reduce**

In [ ]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vector_db.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = "o q é adobe pdf?"

resposta = chat_chain.invoke({'query': pergunta})

print(resposta)

**Refine**

In [64]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vector_db.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = "o q é adobe pdf?"

resposta = chat_chain.invoke({'query': pergunta})

print(resposta['result'])

O Adobe PDF é um formato de arquivo desenvolvido pela Adobe Systems que permite representar documentos de forma independente do aplicativo, hardware e sistema operacional utilizados para criá-los. Ele é amplamente utilizado devido à sua capacidade de preservar a formatação original e ser facilmente compartilhado e visualizado em diferentes dispositivos. Os arquivos PDF também podem ser assinados digitalmente, o que impede qualquer tipo de alteração no arquivo original. Existem vários programas gratuitos disponíveis para gerar arquivos PDF, como PDFMaker, PDF ReDirect e PDFCreator, porém o software mais conhecido para a geração e manipulação de arquivos em PDF é o Adobe Acrobat. É importante ter cuidado para não confundir o Adobe Acrobat com o Adobe Reader, que é simplesmente um leitor gratuito de PDFs e permite apenas a leitura de arquivos PDF.
